In [110]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
import psycopg2
Base = declarative_base()

In [111]:
#Import CSVs and Creating DataFrames
housing = pd.read_csv('reduced_realty_df.csv')
# housing.head()

careers_url = "web_scrape_top-100-careers.html"
careers_mega = pd.read_html(careers_url)
careers = careers_mega[2]
# careers.head()

employment_data = pd.read_excel("state_M2021_dl.xlsx")
# employment_data.head()

In [112]:
housing.columns

Index(['status', 'price', 'city', 'state', 'sold_date', 'house_size'], dtype='object')

In [113]:
#Groupby and Rename Columns
housing_by_state = housing.groupby('state').mean()
housing_by_state = housing_by_state[['price', 'house_size']]
housing_by_state.columns = ['Average_Price', 'Average_Square_Feet']
housing_by_state['Average_Price'] = round(housing_by_state['Average_Price'], 2)
# housing_by_state.head()

In [114]:
employment_df = employment_data[['OCC_TITLE', 'A_MEAN', 'AREA_TITLE']]
# employment_df.head()

In [115]:
# Edit Occupation Names
employment_df = employment_df.replace(to_replace='Dentists, General', value='Dentist')
employment_df = employment_df.replace(to_replace='Registered Nurses', value='Registered Nurse')
employment_df = employment_df.replace(to_replace='Pharmacists', value='Pharmacist')
employment_df = employment_df.replace(to_replace='Computer Systems Analysts', value='Computer Systems Analyst')
employment_df = employment_df.replace(to_replace='Family Medicine Physicians', value='Physician')

In [116]:
#TODO Merge Tables
    # Table 1 Jobs: Job Title, Location, Salary (Range), (Unemployment)
    # Table 2 Housing: Housing Price, Location, Rooms, Acreage, Sq Ft-age
Jobs_df = careers.merge(employment_df, left_on=1, right_on='OCC_TITLE', how='inner')
Jobs_df = Jobs_df[[1, 4, 'A_MEAN', 'AREA_TITLE']]
Jobs_df.columns = ['Occupation', 'National Unemployment Rate', 'State Mean Income', 'State']
# Jobs_df.head()

In [117]:
#TODO Export into PGAdmin4

In [118]:
rds_connection_string = "postgres:postgres@localhost:5432/Employment_ForHousing"
engine = create_engine(f'postgresql://{rds_connection_string}', pool_size=10, max_overflow=20)

In [121]:
#Establishing the connection
conn = psycopg2.connect(
   database="Employment_ForHousing", user='postgres', password='postgres', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Droping table if already exists.
cursor.execute("DROP TABLE IF EXISTS jobs")


#Creating table as per requirement
sql ='''CREATE TABLE jobs(
   id INTEGER NOT NULL PRIMARY KEY,
   Occupation VARCHAR(255),
   National_Unemployment_Rate FLOAT,
   State_Mean_Income FLOAT,
   State VARCHAR(255)
)'''
cursor.execute(sql)
print("Table created successfully........")
conn.commit()
#Closing the connection
conn.close()

Table created successfully........


In [122]:
#Establishing the connection
conn = psycopg2.connect(
   database="Employment_ForHousing", user='postgres', password='postgres', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Droping table if already exists.
cursor.execute("DROP TABLE IF EXISTS housing")


#Creating table as per requirement
sql ='''CREATE TABLE housing(
   id INTEGER NOT NULL PRIMARY KEY,
   Average_Price FLOAT,
   Average_Square_Feet FLOAT,
   State VARCHAR(255)
)'''
cursor.execute(sql)
print("Table created successfully........")
conn.commit()
#Closing the connection
conn.close()

Table created successfully........


In [123]:
# Use pandas to load DataFrame into database
Jobs_df.to_sql(name='jobs', con=engine.connect(), if_exists='replace', index=False) 
housing_by_state.to_sql(name='housing', con=engine.connect(), if_exists='replace', index=False)


In [124]:
pd.read_sql_query("SELECT * FROM jobs", con=engine).head()

,Occupation,National Unemployment Rate,State Mean Income,State
0,Dentist,0.7%,176910,Alabama
1,Dentist,0.7%,168390,Alaska
2,Dentist,0.7%,175030,Arizona
3,Dentist,0.7%,184540,Arkansas
4,Dentist,0.7%,165950,California


In [125]:
pd.read_sql_query("SELECT * FROM housing", con=engine).head()

,Average_Price,Average_Square_Feet
0,427500.45,2031.921645
1,1110065.96,2283.397881
2,596461.02,2269.087580
3,333490.00,1500.000000
4,790566.40,2858.399378
